In [113]:
import yaml
import os
import pandas as pd
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch
from typing import *
import gensim

c:\Users\Димас\venvs\main_venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] Не найдена указанная процедура
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def parse_topic(topic):
    num = int(topic[0])
    s = topic[4:-1].split(' + ')
    w = []
    t = []
    for s_i in s:
        w_i, t_i = s_i.split("*")
        w.append(float(w_i))
        t.append(t_i[1:-1])
    return w, t

In [3]:
dir_path = 'results'

df = []

for file in os.listdir(dir_path):
    if file.endswith('.txt'):
        # print(file[7])
        with open(os.path.join(dir_path, file), 'r', encoding='utf-8') as f:
            df_line = [int(file[16:18]), int(file[19])]
            lines = f.readlines()
            lines = [parse_topic(t) for t in lines]
            # print(lines)
            for topic in lines:
                df_line.append(topic[-1])
            
            if len(df_line) < 7:
                n = 7 - len(df_line)
                df_line += [None] * n
            print(df_line)
            # df.append(pd.Series(df_line), ignore_index=True)
            # df_line = pd.DataFrame(columns=('age', 'gender', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6'), )
            df.append(df_line)

df = pd.DataFrame(df, columns=('age', 'gender', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6'))

[16, 1, ['любить', 'хороший', 'id', 'день', 'любовь', 'друг', 'com', 'любимый', 'http', 'знать'], None, None, None, None]
[16, 2, ['http', 'любить', 'com', 'id', 'друг', 'ru', 'vk', 'день', 'хороший', 'писать'], None, None, None, None]
[17, 1, ['любить', 'любовь', 'день', 'хороший', 'id', 'знать', 'рождение', 'друг', 'тобой', 'человек'], None, None, None, None]
[17, 2, ['любить', 'http', 'com', 'id', 'день', 'друг', 'хороший', 'ru', 'тобой', 'знать'], None, None, None, None]
[18, 1, ['любить', 'день', 'хороший', 'id', 'любовь', 'рождение', 'com', 'http', 'знать', 'друг'], None, None, None, None]
[18, 2, ['http', 'id', 'любить', 'ru', 'день', 'vkontakte', 'друг', 'com', 'хороший', 'новый'], None, None, None, None]
[19, 1, ['любить', 'день', 'хороший', 'любовь', 'com', 'id', 'http', 'рождение', 'человек', '_'], None, None, None, None]
[19, 2, ['любить', 'http', 'день', 'com', 'хороший', 'id', 'любовь', 'друг', 'новый', 'ru'], None, None, None, None]
[20, 1, ['любить', 'день', 'com', 'htt

In [4]:
df.to_csv('topics.csv')

In [115]:
class Encoder:
    def encode_sent(self, tokens: List[str]) -> torch.Tensor:
        pass

    def encode_word_in_context(self,
                               tokens: List[str],
                               target_i: int) -> torch.Tensor:
        pass

class EncoderElmo(Encoder):
    def __init__(self,
                 options_file: str,
                 weight_file: str,
                 requires_grad: bool = False):
        self.elmo = Elmo(options_file,
                         weight_file,
                         num_output_representations=1,
                         requires_grad=requires_grad)
        self.elmo.eval()
        
    def encode_sent(self, tokens: List[str]) -> torch.Tensor:
        out = self.elmo(batch_to_ids([tokens]))["elmo_representations"][0]
        return out[0].mean(dim=0)

    def encode_word_in_context(self,
                               tokens: List[str],
                               target_i: int) -> torch.Tensor:
        out = self.elmo(batch_to_ids([tokens]))["elmo_representations"][0]
        return out[0, target_i]
    
class EncoderWord2Vec(Encoder):
    def __init__(self, path: str):
        self.wv = gensim.models.keyedvectors.FastTextKeyedVectors.load(path)

    def encode_sent(self, tokens: List[str]) -> torch.Tensor:
        return torch.stack([torch.from_numpy(self.wv.get_vector(token))
                            for token in tokens]).mean(dim=0)

    def encode_word_in_context(self,
                               tokens: List[str],
                               target_i: int) -> torch.Tensor:
        emb = torch.stack([torch.from_numpy(self.wv.get_vector(token))
                           for i, token in enumerate(tokens)])
        # return torch.cat((emb[target_i], emb.mean(dim=0)))
        return emb.mean(dim=0)

In [6]:
# path = "ruwikiruscorpora_tokens_elmo_1024_2019"
# elmo = EncoderElmo(
#     options_file=os.path.join(path, "options.json"),
#     weight_file=os.path.join(path, "model.hdf5"),
# )

In [5]:
path = "geowac_tokens_none_fasttextskipgram_300_5_2020"
wv = EncoderWord2Vec(os.path.join(path, "model.model"))

In [ ]:
df = pd.read_csv('topics.csv')

In [6]:
df

,age,gender,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6
0,16,1,"[любить, хороший, id, день, любовь, друг, com,...",None,None,None,None,None
1,16,2,"[http, любить, com, id, друг, ru, vk, день, хо...",None,None,None,None,None
2,17,1,"[любить, любовь, день, хороший, id, знать, рож...",None,None,None,None,None
3,17,2,"[любить, http, com, id, день, друг, хороший, r...",None,None,None,None,None
4,18,1,"[любить, день, хороший, id, любовь, рождение, ...",None,None,None,None,None
...,...,...,...,...,...,...,...,...
168,42,1,"[помогать, девочка, нужный, искать, месяц, соб...","[друг, любить, день, любовь, хороший, человек,...","[com, https, vk, http, _, ru, id, amp, фото, м...","[видео, кожа, лицо, продукт, онлаин, бизнес, в...","[цена, размер, см, наличие, цвет, качество, ку...","[класс, писать, город, группа, проходить, рабо..."
169,60,1,"[мамочка, любовь, душа, каждый, жить, человек,...","[http, com, ru, рождение, здоровье, https, vk,...","[ребенок, помогать, мама, большой, детея, тако...","[писать, видео, класс, город, фото, первый, вс...","[рецепт, купить, простой, минута, вкусный, сал...","[любить, день, хороший, друг, новый, дорогой, ..."
170,61,1,"[рецепт, простой, вкусный, фото, готовить, сал...","[день, любовь, друг, рождение, любить, пусть, ...","[com, vk, http, ru, https, app, отправлять, _,...","[пирог, цветок, красота, класс, лето, весна, и...","[тесто, вкусно, домашний, девочка, масло, пиро...","[человек, помогать, какой, видео, сеичас, ребе..."
171,61,2,"[страна, новость, изображение, президент, укра...","[заголовок, город, май, область, проходить, пр...","[день, новый, друг, человек, хороший, gt, праз...","[видео, получать, россииский, сообщать, резуль...","[http, ru, com, art, https, press, _ , feed, w...","[армения, любовь, любить, жить, пусть, знать, ..."


In [10]:
wv.wv.most_similar(positive=df.loc[0].topic_1, topn=10)

[('интерестный', 0.6517411470413208),
 ('благодарный', 0.6117192506790161),
 ('комментарийлучший', 0.6030017137527466),
 ('интерестно', 0.6007556319236755),
 ('интерессно', 0.5992611646652222),
 ('ведь', 0.5982648730278015),
 ('понимающий', 0.5982488989830017),
 ('замечательный', 0.5978759527206421),
 ('любящий', 0.5974389910697937),
 ('обожать', 0.593932569026947)]

In [37]:
df.loc[0].topic_1

['любить',
 'хороший',
 'id',
 'день',
 'любовь',
 'друг',
 'com',
 'любимый',
 'http',
 'знать']

# Словарь РуСентиЛекс

In [1]:
from collections import defaultdict
import numpy as np
import pandas as pd


def get_sent(line):
    if line.find('positive/negative') > 0:
        return'positive/negative'
    if line.find('negative') > 0 or line.find('negaitve') > 0:
        return'negative'
    if line.find('positive') > 0 or line.find('postive') > 0 or line.find('postitive') > 0 or line.find('posititve') > 0 or line.find('posiive') > 0:
        return'positive'
    if line.find('neutral') > 0:
        return'neutral'

with open('rusentilex_2017.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()[18:]
    lines = [(get_sent(i), i.split(', ')[0]) for i in lines]
    df = pd.DataFrame(lines).rename(columns={0 : 'sent', 1 : 'word'})
    df['n'] = pd.Series(np.ones(len(df)))
    df = df.groupby(['sent', 'word']).count().reset_index()
    df['mn'] = df.groupby('word')['n'].transform(max)
    df =df[df['mn'] == df['n']]
    df = df.groupby('word').min().reset_index()[['word', 'sent']]
    df = df[df['word'].apply(lambda x: len(x.split(' ')) == 1)]
    sentiment_corpus = defaultdict(list)
    for word, sent in df.values:
        sentiment_corpus[sent].append(word)

sentiment_corpus['negative'].append(sentiment_corpus['positive/negative'])
sentiment_corpus.pop('positive/negative')

for key in sentiment_corpus.keys():
    print(f'{key}: len = {len(sentiment_corpus[key])}, example = {sentiment_corpus[key][:5]}')

negative: len = 7768, example = ['аборт', 'абортивный', 'абракадабра', 'абсурд', 'абсурдность']
positive: len = 2450, example = ['авторитет', 'авторитетность', 'авторитетный', 'адаптационный', 'адаптация']
neutral: len = 498, example = ['байка', 'балласт', 'бархатный', 'безделица', 'безделка']


- Словосочетания удаляем
- Выбираем для слова ту метку, которой больше.
- При равных количествах меток приоритет такой: positive -> neutral -> negative
- positive/negative целиком трансформируется в negative

# Поиск синонимов 

In [116]:
path = "geowac_tokens_none_fasttextskipgram_300_5_2020"
wv = EncoderWord2Vec(os.path.join(path, "model.model"))

In [127]:
words = list(neg_set)[13]
print(f'слово: {words}')
print(f'ближайшие синонимы:')
wv.wv.most_similar(words)

слово: дискомфортность
ближайшие синонимы:


[('комфортность', 0.8312166929244995),
 ('дискомфортных', 0.8022353053092957),
 ('дискомфортные', 0.7948716878890991),
 ('дискомфорт', 0.7768241763114929),
 ('дискомфорта', 0.7711535096168518),
 ('дискомфортно', 0.7632939219474792),
 ('дискомфорту', 0.7564236521720886),
 ('дискомфортом', 0.7393903732299805),
 ('дискомфорте', 0.7393408417701721),
 ('болезненность', 0.7377340197563171)]

In [ ]:
a = pd.DataFrame([
    [1, 'positive'],
    [1, 'negative'],
    [1, 'negative'],
    [1, 'positive'],
    [1, 'positive'],
    [1, 'positive'],
    ])

СЛОВАРЬ РУССКИХ СИНОНИМОВ И СХОДНЫХ ПО СМЫСЛУ ВЫРАЖЕНИЙ
Синонимы (от греческого слова «одноимённый») — это слова, совпадающие или близкие по значению. Они выражают одно и то же понятие, но различаются оттенками значения, например: занавесь (общее наименование: полотнище ткани, которым завешивают окна, двери, картины и т.п., отгораживают часть помещения), занавес (большое тяжёлое полотнище, закрывающее дверь, проём и т.п., отгораживающее часть помещения, сцену), занавеска (полотнище из легкой ткани), штора (полотнище ткани для завешивания окна), гардина и портьера (полотнище ткани для завешивания двери). 
Синонимы могут также различаться по выразительности, могут быть книжными и разговорными, современными и устарелыми, просторечными, народно - поэтическими, ласкательными и шутливыми: мальчик (подросток мужского пола), мальчишка (разговорное), мальчуган и мальчонка (разговорное и ласкательное), мальчонок (просторечное, ласкательное), парнишка и паренёк (разговорное и ласкательно-фамильярное), хлопец и хлопчик (просторечное и областное), шпингалет (просторечное), пацан (грубоватопросторечное), шкет (жаргонное). 
Синонимы отделяются друг от друга запятой; точка с запятой разделяет синонимы с разными оттенками значения.

Примеры условных обозначений:                      
        Ср. <Ясный> — сравни со словарной статьей «ясный»           
        Ср. <Ад, Много и Углубление> сравни со статьями «ад», «много» и «углубление»                                   
        Прот. <Райский> — Противоположность статье «Райский»        
        [нем. Abschied отставка, увольнение] — пояснение смысла слова 
        || брать авансом, забирать авансом — устойчивые (типичные) словосочетания, идиомы  
        Пушк. — А. Пушкин, Тург., Тургенев — И.Тургенев и т.д.       
        (Даль, алкать) — словарь В.Даля, статья АЛКАТЬ               


In [8]:
with open('n-abramov-slovar-russkih-sinonimov-i-shodnyh-po-smyslu-vyrazheniy.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [9]:
lines = [i[:-1].split('см.') for i in lines]
lines

[['абдериты ', ' глупый'],
 ['абдикация [отречение; отказ от сана и власти правителя, сложение с себя этого звания (Даль)] ',
  ' отказ'],
 ['абзац ', ' часть'],
 ['абитуриент ', ' ученик'],
 ['Абонировать, нанимать, брать внаем (внаймы). Абонировать ложу, кресло.'],
 ['      Абонироваться на ложу, на кресло, на чтение книг в библиотеке,'],
 ['      подписаться на журнал, выписать газету. Ср. <Нанимать>. См. нанимать'],
 ['абонироваться ', ' абонировать, нанимать, подписываться'],
 ['абордаж ', ' столкновение'],
 ['абориген ', ' житель, туземец'],
 ['абортировать ', ' рождать'],
 ['абракадабра ', ' нелепость'],
 ['абрис ', ' очерк'],
 ['абсентеизм [уклонение от посещений, связанных с выполнением каких-л. общественных обязанностей (Ушаков)] ',
  ' отсутствие'],
 ['Абсолютный, полный, совершенный, безотносительный, самостоятельный.'],
 ['      Абсолютное большинство голосов -- перевес над противною стороною хотя'],
 ['      бы одним голосом. Абсолютный вес. Абсолютная высота места (т. е.